In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
# This is the cloud masking function provided by GEE but adapted for use in Python.
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    mask = mask.bitwiseAnd(cirrusBitMask).eq(0)

    return image.updateMask(mask).divide(10000)


# Define the geometry of the area for which you would like images.
geoms = ee.Geometry.Polygon([[33.8777, -13.4055],
                            [33.8777, -13.3157],
                            [33.9701, -13.3157],
                            [33.9701, -13.4055]])

# Call collection of satellite images.
collection = (ee.ImageCollection("COPERNICUS/S2_SR")
              # Filter for images within a given date range.
              .filter(ee.Filter.date('2017-01-01', '2017-03-31'))
              # Filter for images that overlap with the assigned geometry.
              .filterBounds(geom)
              # Filter for images that have less then 20% cloud coverage.
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
              # Apply cloud mask.
              .map(maskS2clouds)
             )

# Sort images in the collection by index (which is equivalent to sorting by date), 
# with the oldest images at the front of the collection.
# Convert collection into a single image mosaic where only images at the top of the collection are visible.
image = collection.sort('system:index', opt_ascending=False)


# Assign visualization parameters to the image.
image = image.visualize(bands=['B4', 'B3', 'B2'],
                        min=[0.0, 0.0, 0.0],
                        max=[0.3, 0.3, 0.3]
                       )

# Assign export parameters.
task_config = {
    'region': geom.coordinates().getInfo(),
    'folder': 'Example_Folder_Name',
    'scale': 10,
    'crs': 'EPSG:4326',
    'description': 'Example_File_Name'
}

# Export Image
task = ee.batch.Export.image.toDrive(my_image, **task_config)
task.start()

In [ ]:
geometry = ee.Geometry.Polygon(
        [[[-91.88241557579721, 34.44043152853586],
          [-91.87939748709968, 34.36609809217062],
          [-91.77798133624677, 34.36664970703428],
          [-91.78038630176351, 34.44098250970811]]])
geometry2 = ee.Geometry.Polygon(
        [[[-91.4533577608951, 33.552587560716816],
          [-91.45037247722163, 33.478254083278415],
          [-91.35000778380517, 33.47880573492943],
          [-91.35238925112668, 33.5531385048565]]])
geometry3 = ee.Geometry.Polygon(
        [[[-89.15098738346563, 36.13552640132718],
          [-89.14790252766808, 36.06119304592777],
          [-89.04434623756553, 36.061744588329724],
          [-89.04679903547047, 36.136077455449566]]])

geometry4 = ee.Geometry.Polygon(
        [[[-90.88553982261615, 35.30300407163324],
          [-90.88248845682877, 35.22867067603548],
          [-90.78000564911692, 35.22922225440435],
          [-90.78243445454433, 35.30355508954531]]])

geometry5 = ee.Geometry.Polygon(
        [[[-97.5896807191784, 37.07211128420012],
          [-97.58655650141277, 36.997777975079025],
          [-97.48173859013154, 36.99832947609662],
          [-97.48421958307605, 37.072662379989104]]])

geoms = [geometry, geometry2, ]